In [7]:
import pandas as pd
import sqlalchemy as sql
import numpy as np
import time 

connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')

query = "SELECT * FROM `분기순서`"
분기순서 = pd.read_sql_query(query, sql_engine)
최근분기 = (분기순서.iloc[0]['년도'], 분기순서.iloc[0]['분기'])

if 최근분기[1] == 4:
    최근년도 = 최근분기[0]
else:
    최근년도 = 최근분기[0]-1
    
query = "SELECT * FROM `종목별가치지표`"
company_df = pd.read_sql_query(query, sql_engine)

query = "SELECT * FROM `재무데이터통합_분기` WHERE `년도` = {} AND `분기` = {}".format(최근분기[0], 최근분기[1])
df = pd.read_sql_query(query, sql_engine)

query = "SELECT * FROM `재무데이터통합_연간` WHERE `년도` = {}".format(최근년도)
df_year = pd.read_sql_query(query, sql_engine)

query = "SELECT DISTINCT `날짜` FROM `일봉데이터` ORDER BY `날짜` DESC"
날짜순서 = pd.read_sql_query(query, sql_engine)
최근날짜 = 날짜순서.iloc[0,0]

query = "SELECT `날짜`, `종목코드`, `종가`, `시가총액` FROM `일봉데이터` WHERE `날짜` = '{}'".format(최근날짜)
가격_df = pd.read_sql_query(query, sql_engine)

query = "SELECT * FROM `FnGuide_컨센서스_연간데이터`"
컨센서스연간 = pd.read_sql_query(query, sql_engine)

query = "SELECT * FROM `NAVER_FINANCE_SUMMARY_연간데이터`"
Summary연간 = pd.read_sql_query(query, sql_engine)

def 별도지배주주순이익(ser, value):
    if value == '(지배주주지분)당기순이익':
        if ser['회계'] == 'IFRS연결':
            return value
        elif ser['회계'] == 'KIFRS별도':
            return '당기순이익'
        else: 
            return value
    else:
        return value
    
def Get_value(ser, value):
    try:
        ser[별도지배주주순이익(ser, value)]
        return ser[별도지배주주순이익(ser, value)]
    except:
        return np.nan

# 수정한 DF_selector
def DF_selecter(df, 종목코드, 년도, 분기, n):
    if np.isnan(분기):
        Sel_DF = df.loc[df['종목코드'] == 종목코드].loc[df['년도'] == 년도]
    else:
        Sel_DF = df.loc[df['종목코드'] == 종목코드].loc[df['년도'] == 년도].loc[df['분기'] == 분기]
    if not Sel_DF.empty:
        Sel_DF_연결 = Sel_DF.loc[Sel_DF['회계'] == 'IFRS연결']
        Sel_DF_별도 = Sel_DF.loc[Sel_DF['회계'] == 'KIFRS별도']
        if not Sel_DF_연결.empty:
            if not Sel_DF_연결.iloc[0][n:].isnull().all():
                return Sel_DF_연결.iloc[0]
            elif not Sel_DF_별도.empty:
                if not Sel_DF_별도.iloc[0][n:].isnull().all():
                    return Sel_DF_별도.iloc[0]
        elif not Sel_DF_별도.empty:
            if not Sel_DF_별도.iloc[0][n:].isnull().all():
                return Sel_DF_별도.iloc[0]
        else:
            return pd.Series([],dtype=pd.StringDtype())  
    else:
        return pd.Series([],dtype=pd.StringDtype())

def DF_year_selecter(df, 종목코드, 년도, 회계):
    Sel_DF = df.loc[df['종목코드'] == 종목코드].loc[df['회계'] == 회계].loc[df['년도'] == 년도]
    if not Sel_DF.empty:
        return Sel_DF.iloc[0]
    else:
        return pd.Series([],dtype=pd.StringDtype()) 

def 증가율(최근값, 이전값):
    return (최근값-이전값) * 100/이전값

total_list = []
for i in range(0, len(company_df)):
    종목코드 = company_df.iloc[i]['종목코드']
    종목명 = company_df.iloc[i]['종목명']
    
    최근분기재무 = DF_selecter(df, 종목코드, 최근분기[0], 최근분기[1], 8)
    try:
        회계 = 최근분기재무['회계']
    except:
        회계 = np.nan
        
    최근년도재무 = DF_selecter(df_year, 종목코드, 최근년도, np.nan, 8)
    try:
        연간회계 = 최근년도재무['회계']
    except:
        연간회계 = np.nan
        
    컨센서스1Y = DF_year_selecter(컨센서스연간, 종목코드, 최근년도+1, 회계)
    컨센서스2Y = DF_year_selecter(컨센서스연간, 종목코드, 최근년도+2, 회계)
    최근년도Summary = DF_year_selecter(Summary연간, 종목코드, 최근년도, 연간회계)
        
    if not 가격_df.loc[가격_df['종목코드'] == 종목코드].empty:
        가격정보 = 가격_df.loc[가격_df['종목코드'] == 종목코드].iloc[0]
        종가 = 가격정보['종가']
        시가총액 = 가격정보['시가총액']
        if 시가총액 == None:
            시가총액 = np.nan
        if 종가 == None:
            종가 = np.nan
    else:
        종가 = np.nan
        시가총액 = np.nan
    
    최근분기지배주주당기순이익 = Get_value(최근분기재무, '(지배주주지분)당기순이익')
    최근분기유동자산 = Get_value(최근분기재무, '유동자산')
    최근분기부채총계 = Get_value(최근분기재무, '부채총계')
    최근분기지배주주자본 = Get_value(최근분기재무, '지배주주지분')
    
    최근년도_EPS = Get_value(최근년도재무, '(지배주주지분)주당순이익')
    최근년도영업활동현금흐름 = Get_value(최근년도재무, '영업활동으로인한현금흐름')
    최근년도매출액 = Get_value(최근년도재무, '매출액(수익)')
    최근년도지배주주당기순이익 = Get_value(최근년도재무, '(지배주주지분)당기순이익')
    최근년도FCFF = company_df.iloc[i]['FCFF']
    최근년도지배주주자본 = Get_value(최근년도재무, '지배주주지분')
    최근년도_DPS = Get_value(최근년도Summary, '현금DPS(원)')
    
    Trailing_지배주주순이익 = company_df.iloc[i]['Trailing_지배주주순이익']
    Trailing_매출액 = company_df.iloc[i]['Trailing_매출액']
    
    컨센서스1Y_EPS = Get_value(컨센서스1Y, 'EPS(원)')
    컨센서스1Y_지배주주순이익 = Get_value(컨센서스1Y, '지배주주순이익(억원)')
    컨센서스2Y_지배주주순이익 = Get_value(컨센서스2Y, '지배주주순이익(억원)')
    
    EPS_증가율_1년뒤 = 증가율(컨센서스1Y_EPS, 최근년도_EPS)
    
    PCR = 시가총액/최근년도영업활동현금흐름
    PSR = 시가총액/최근년도매출액
    PBR = 시가총액/최근년도지배주주자본
    PER = 시가총액/최근년도지배주주당기순이익
    PFCR = 시가총액/최근년도FCFF
    NCAV = (최근분기유동자산-최근분기부채총계)/시가총액
    Trailing_PER = 시가총액/Trailing_지배주주순이익
    Trailing_PSR = 시가총액/Trailing_매출액
    Trailing_PBR = 시가총액/최근분기지배주주자본
    Forward_PER_올해 = 시가총액/컨센서스1Y_지배주주순이익
    Forward_PER_1년후 = 시가총액/컨센서스2Y_지배주주순이익
    분기_PER = 시가총액/(최근분기지배주주당기순이익*4)
    PEG = Trailing_PER/EPS_증가율_1년뒤
    시가배당률 = (최근년도_DPS/종가) * 100
    
    result_list = [종목코드, 종목명, 회계, 최근날짜, PCR, PSR, PBR, PER, PFCR, NCAV, 분기_PER, Trailing_PER, Trailing_PSR, Trailing_PBR, 
                  Forward_PER_올해, Forward_PER_1년후, PEG, 시가배당률]
    total_list.append(result_list)

result_df = pd.DataFrame(total_list, columns = ['종목코드', '종목명', '회계', '최근날짜', 'PCR', 'PSR', 'PBR', 'PER', 'PFCR', 'NCAV', 
                                    '분기_PER', 'Trailing_PER', 'Trailing_PSR', 'Trailing_PBR', 'Forward_PER_올해', 'Forward_PER_1년후', 'PEG', '시가배당률'])

result_df.insert(loc=0, column='No', value=np.nan)
result_df = result_df.replace([np.inf, -np.inf], np.nan)

No_list = []
for i in range(0, len(result_df)):
    No_list.append(i+1)
result_df['No'] = No_list

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '회계': sql.types.VARCHAR(7), 
    '날짜': sql.types.DateTime()
}

for i in list(result_df.columns)[5:]:
    dtype[i] = sql.types.FLOAT() 
    
result_df.to_sql(name="종목별가격지표", con=sql_engine, if_exists='replace', index=False, 
         dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `종목별가격지표` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `종목별가격지표` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `종목별가격지표` ADD INDEX(`종목코드`);')
    
now = time.localtime()
current_time = "%04d-%02d-%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

print("[" + current_time + "] " + "종목별가격지표 테이블 "+ str(최근날짜) + " 기준으로 업데이트 되었습니다.")

#result_df


,No,종목코드,종목명,회계,최근날짜,PCR,PSR,PBR,PER,PFCR,NCAV,Trailing_PER,Trailing_PSR,Trailing_PBR,Forward_PER_올해,Forward_PER_1년후,PEG
0,1,000020,동화약품,IFRS연결,2020-12-01,41.629131,1.800563,1.838115,58.820407,-6.998454,0.277919,19.275567,1.928686,1.698535,NaN,NaN,NaN
1,2,000040,KR모터스,IFRS연결,2020-12-01,-16.963050,0.905694,6.476137,-2.966407,-1.525247,-0.343749,-4.118030,0.956316,2.895072,NaN,NaN,NaN
2,3,000050,경방,IFRS연결,2020-12-01,4.919350,0.956646,0.443628,7.542575,-4.174503,-1.164714,7.279528,0.978158,0.444946,NaN,NaN,NaN
3,4,000060,메리츠화재,NaN,2020-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,000070,삼양홀딩스,IFRS연결,2020-12-01,2.613037,0.229889,0.402987,18.530768,-0.481090,-0.471707,13.458985,0.233522,0.391231,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500,2501,053080,원방테크,IFRS연결,2020-12-01,20.195612,0.812263,1.871441,13.312689,-5.572086,0.145520,NaN,NaN,1.376890,7.332292,5.000189,NaN
2501,2502,348210,넥스틴,IFRS연결,2020-12-01,-430.228926,33.573000,219.258044,-121.402917,-85.285464,0.101619,NaN,NaN,8.089448,20.216154,13.893040,NaN
2502,2503,352820,빅히트,NaN,2020-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2503,2504,304840,피플바이오,NaN,2020-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
import pandas as pd
import sqlalchemy as sql
import numpy as np
import time 

connect_string = 'mysql://quantk:znjsxm!!@@DB12@localhost:3377/quantk?charset=utf8'
sql_engine = sql.create_engine(connect_string, encoding='utf-8')

query = "SELECT * FROM `분기순서`"
분기순서 = pd.read_sql_query(query, sql_engine)
최근분기 = (분기순서.iloc[0]['년도'], 분기순서.iloc[0]['분기'])

if 최근분기[1] == 4:
    최근년도 = 최근분기[0]
else:
    최근년도 = 최근분기[0]-1
    
query = "SELECT * FROM `종목별가치지표`"
company_df = pd.read_sql_query(query, sql_engine)

query = "SELECT * FROM `재무데이터통합_분기` WHERE `년도` = {} AND `분기` = {}".format(최근분기[0], 최근분기[1])
df = pd.read_sql_query(query, sql_engine)

query = "SELECT * FROM `재무데이터통합_연간` WHERE `년도` = {}".format(최근년도)
df_year = pd.read_sql_query(query, sql_engine)

query = "SELECT DISTINCT `날짜` FROM `v일봉데이터` ORDER BY `날짜` DESC"
날짜순서 = pd.read_sql_query(query, sql_engine)
최근날짜 = 날짜순서.iloc[0,0]

query = "SELECT `날짜`, `종목코드`, `종가`, `시가총액` FROM `v일봉데이터` WHERE `날짜` = '{}'".format(최근날짜)
가격_df = pd.read_sql_query(query, sql_engine)

query = "SELECT * FROM `FnGuide_컨센서스_연간데이터`"
컨센서스연간 = pd.read_sql_query(query, sql_engine)
   
def Get_value(ser, value):
    try:
        ser[value]
        return ser[value]
    except:
        return np.nan
    
def DF_selecter(df, 종목코드, 년도, 분기):
    Sel_DF = df.loc[df['종목코드'] == 종목코드].loc[df['년도'] == 년도].loc[df['분기'] == 분기]
    if not Sel_DF.empty:
        if not Sel_DF.loc[Sel_DF['회계'] == 'IFRS연결'].iloc[0][8:].isnull().all():
            return Sel_DF.loc[Sel_DF['회계'] == 'IFRS연결'].iloc[0]
        elif not Sel_DF.loc[Sel_DF['회계'] == 'KIFRS별도'].iloc[0][8:].isnull().all():
            return Sel_DF.loc[Sel_DF['회계'] == 'KIFRS별도'].iloc[0]
        else:
            return pd.Series([],dtype=pd.StringDtype())  
    else:
        return pd.Series([],dtype=pd.StringDtype())  

def DF_year_selecter(df, 종목코드, 년도, 회계):
    Sel_DF = df.loc[df['종목코드'] == 종목코드].loc[df['회계'] == 회계].loc[df['년도'] == 년도]
    if not Sel_DF.empty:
        return Sel_DF.iloc[0]
    else:
        return pd.Series([],dtype=pd.StringDtype()) 

def 증가율(최근값, 이전값):
    return (최근값-이전값) * 100/이전값

total_list = []
for i in range(0, len(company_df)):
#for i in range(0, 10):
    종목코드 = company_df.iloc[i]['종목코드']
    종목명 = company_df.iloc[i]['종목명']
    
    최근분기재무 = DF_selecter(df, 종목코드, 최근분기[0], 최근분기[1])
    try:
        회계 = 최근분기재무['회계']
    except:
        회계 = np.nan
        
    최근년도재무 = DF_year_selecter(df_year, 종목코드, 최근년도, 회계)
    최근년도컨센서스데이터 = DF_year_selecter(컨센서스연간, 종목코드, 최근년도, 회계)
    컨센서스1Y = DF_year_selecter(컨센서스연간, 종목코드, 최근년도+1, 회계)
    컨센서스2Y = DF_year_selecter(컨센서스연간, 종목코드, 최근년도+2, 회계)
    
    if not 가격_df.loc[가격_df['종목코드'] == 종목코드].empty:
        가격정보 = 가격_df.loc[가격_df['종목코드'] == 종목코드].iloc[0]
        종가 = 가격정보['종가']
        시가총액 = 가격정보['시가총액']
        if 시가총액 == None:
            시가총액 = np.nan
        if 종가 == None:
            종가 = np.nan
    else:
        종가 = np.nan
        시가총액 = np.nan
    

    #최근분기영업활동현금흐름 = Get_value(최근분기재무, '영업활동으로인한현금흐름')
    #최근분기매출액 = Get_value(최근분기재무, '매출액(수익)')
    #최근분기자본총계 = Get_value(최근분기재무, '자본총계')
    최근분기지배주주당기순이익 = Get_value(최근분기재무, '(지배주주지분)당기순이익')
    #최근분기FCFF = Get_value(최근분기재무, 'FCFF')
    최근분기유동자산 = Get_value(최근분기재무, '유동자산')
    최근분기부채총계 = Get_value(최근분기재무, '부채총계')
    최근분기지배주주자본 = Get_value(최근분기재무, '지배주주지분')
    
    최근년도_EPS = Get_value(최근년도재무, '(지배주주지분)주당순이익')
    최근년도영업활동현금흐름 = Get_value(최근년도재무, '영업활동으로인한현금흐름')
    최근년도매출액 = Get_value(최근년도재무, '매출액(수익)')
    #최근년도자본총계 = Get_value(최근년도재무, '자본총계')
    최근년도지배주주당기순이익 = Get_value(최근년도재무, '(지배주주지분)당기순이익')
    최근년도FCFF = company_df.iloc[i]['FCFF']
    최근년도지배주주자본 = Get_value(최근년도재무, '지배주주지분')
    최근년도_DPS = Get_value(최근년도컨센서스데이터, 'DPS(원)')
    
    Trailing_지배주주순이익 = company_df.iloc[i]['Trailing_지배주주순이익']
    Trailing_매출액 = company_df.iloc[i]['Trailing_매출액']
    
    컨센서스1Y_EPS = Get_value(컨센서스1Y, 'EPS(원)')
    컨센서스1Y_지배주주순이익 = Get_value(컨센서스1Y, '지배주주순이익(억원)')
    컨센서스2Y_지배주주순이익 = Get_value(컨센서스2Y, '지배주주순이익(억원)')
    
    EPS_증가율_1년뒤 = 증가율(컨센서스1Y_EPS, 최근년도_EPS)
    
    PCR = 시가총액/최근년도영업활동현금흐름
    PSR = 시가총액/최근년도매출액
    PBR = 시가총액/최근년도지배주주자본
    PER = 시가총액/최근년도지배주주당기순이익
    PFCR = 시가총액/최근년도FCFF
    NCAV = (최근분기유동자산-최근분기부채총계)/시가총액
    Trailing_PER = 시가총액/Trailing_지배주주순이익
    Trailing_PSR = 시가총액/Trailing_매출액
    Trailing_PBR = 시가총액/최근분기지배주주자본
    Forward_PER_올해 = 시가총액/컨센서스1Y_지배주주순이익
    Forward_PER_1년후 = 시가총액/컨센서스2Y_지배주주순이익
    분기_PER = 시가총액/(최근분기지배주주당기순이익*4)
    PEG = Trailing_PER/EPS_증가율_1년뒤
    시가배당률 = (최근년도_DPS/종가) * 100
    
    result_list = [종목코드, 종목명, 회계, 최근날짜, PCR, PSR, PBR, PER, PFCR, NCAV, 분기_PER, Trailing_PER, Trailing_PSR, Trailing_PBR, 
                  Forward_PER_올해, Forward_PER_1년후, PEG, 시가배당률]
    total_list.append(result_list)

result_df = pd.DataFrame(total_list, columns = ['종목코드', '종목명', '회계', '최근날짜', 'PCR', 'PSR', 'PBR', 'PER', 'PFCR', 'NCAV', 
                                    '분기_PER', 'Trailing_PER', 'Trailing_PSR', 'Trailing_PBR', 'Forward_PER_올해', 'Forward_PER_1년후', 'PEG', '시가배당률'])

result_df.insert(loc=0, column='No', value=np.nan)
result_df = result_df.replace([np.inf, -np.inf], np.nan)

No_list = []
for i in range(0, len(result_df)):
    No_list.append(i+1)
result_df['No'] = No_list

dtype = {
   'No': sql.types.BIGINT(),
    '종목코드': sql.types.VARCHAR(6), 
    '종목명': sql.types.VARCHAR(50), 
    '회계': sql.types.VARCHAR(7), 
    '날짜': sql.types.DateTime()
}

for i in list(result_df.columns)[5:]:
    dtype[i] = sql.types.FLOAT() 
    
result_df.to_sql(name="종목별가격지표", con=sql_engine, if_exists='replace', index=False, 
         dtype = dtype)

with sql_engine.connect() as con:
    con.execute('ALTER TABLE `종목별가격지표` ADD PRIMARY KEY(`No`);')
    con.execute('ALTER TABLE `종목별가격지표` CHANGE `No` `No` BIGINT(20) NOT NULL AUTO_INCREMENT;')
    con.execute('ALTER TABLE `종목별가격지표` ADD INDEX(`종목코드`);')
    
now = time.localtime()
current_time = "%04d-%02d-%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

print("[" + current_time + "] " + "종목별가격지표 테이블 "+ str(최근날짜) + " 기준으로 업데이트 되었습니다.")

#result_df


<ipython-input-2-e3c271444bd5>:127: RuntimeWarning: divide by zero encountered in double_scalars
  PFCR = 시가총액/최근년도FCFF
<ipython-input-2-e3c271444bd5>:127: RuntimeWarning: divide by zero encountered in double_scalars
  PFCR = 시가총액/최근년도FCFF
<ipython-input-2-e3c271444bd5>:127: RuntimeWarning: divide by zero encountered in double_scalars
  PFCR = 시가총액/최근년도FCFF
<ipython-input-2-e3c271444bd5>:127: RuntimeWarning: divide by zero encountered in double_scalars
  PFCR = 시가총액/최근년도FCFF
<ipython-input-2-e3c271444bd5>:127: RuntimeWarning: divide by zero encountered in double_scalars
  PFCR = 시가총액/최근년도FCFF
<ipython-input-2-e3c271444bd5>:127: RuntimeWarning: divide by zero encountered in double_scalars
  PFCR = 시가총액/최근년도FCFF
<ipython-input-2-e3c271444bd5>:127: RuntimeWarning: divide by zero encountered in double_scalars
  PFCR = 시가총액/최근년도FCFF
<ipython-input-2-e3c271444bd5>:127: RuntimeWarning: divide by zero encountered in double_scalars
  PFCR = 시가총액/최근년도FCFF
<ipython-input-2-e3c271444bd5>:124: Runt

<ipython-input-2-e3c271444bd5>:124: RuntimeWarning: divide by zero encountered in double_scalars
  PSR = 시가총액/최근년도매출액
<ipython-input-2-e3c271444bd5>:126: RuntimeWarning: divide by zero encountered in double_scalars
  PER = 시가총액/최근년도지배주주당기순이익
<ipython-input-2-e3c271444bd5>:134: RuntimeWarning: divide by zero encountered in double_scalars
  분기_PER = 시가총액/(최근분기지배주주당기순이익*4)
<ipython-input-2-e3c271444bd5>:124: RuntimeWarning: divide by zero encountered in double_scalars
  PSR = 시가총액/최근년도매출액
<ipython-input-2-e3c271444bd5>:134: RuntimeWarning: divide by zero encountered in double_scalars
  분기_PER = 시가총액/(최근분기지배주주당기순이익*4)
<ipython-input-2-e3c271444bd5>:124: RuntimeWarning: divide by zero encountered in double_scalars
  PSR = 시가총액/최근년도매출액
<ipython-input-2-e3c271444bd5>:126: RuntimeWarning: divide by zero encountered in double_scalars
  PER = 시가총액/최근년도지배주주당기순이익
<ipython-input-2-e3c271444bd5>:134: RuntimeWarning: divide by zero encountered in double_scalars
  분기_PER = 시가총액/(최근분기지배주주당기순이익*4)
<ipyt

[2020-12-09 13:41:27] 종목별가격지표 테이블 2020-12-09 기준으로 업데이트 되었습니다.
